In [ ]:
import numpy as np
import cupy as cp
import sys
import pandas as pd
import time
import matplotlib.pyplot as plt

In [ ]:
gpu = 0
cp.cuda.Device(gpu).use()
show = True
domain='space'
size_v = 48
size_1 = 164# 144

n = 800  # object size in each dimension
npsi = n
pad = 128#n//8 # pad for the reconstructed probe
npos = 16 # total number of positions
z1 = 12.75e-3 # [m] position of the CA
detector_pixelsize = 3.03751e-6
energy = 33.35  # [keV] xray energy
wavelength = 1.24e-09/energy  # [m] wave length
focusToDetectorDistance = 1.28  # [m]
z2 = focusToDetectorDistance-z1
distance = (z1*z2)/focusToDetectorDistance
magnification = focusToDetectorDistance/z1
voxelsize = np.abs(detector_pixelsize/magnification)  # object voxel size

extra = 8
nobj = 3072+512+2*pad
nprb = n+2*pad
npatch = nprb+2*extra

flg = f'{domain}_{size_v}_{size_1}'

path = f'/data/vnikitin/ESRF/ID16A/20240924/SiemensLH/code2um_nfp18x18_01'
path_out = f'/data/vnikitin/ESRF/ID16A/20240924_rec2/SiemensLH/code2um_nfp18x18_01'

In [ ]:
wavelength,voxelsize,distance

In [ ]:
fx = cp.fft.fftfreq(2*npsi, d=voxelsize).astype("float32")
[fx, fy] = cp.meshgrid(fx, fx)
unimod = np.exp(1j * 2*np.pi* distance /wavelength)
fker = (cp.exp(-1j * cp.pi * wavelength * distance * (fx**2 + fy**2))).astype('complex64')
ker = cp.fft.fftshift(cp.fft.ifft2(fker))#*n**2#/npsi/np.pi*2*npsi**2


size_v = 64#*n//512
size_1 = 150#*n//512# 144
x = cp.arange(-npsi,npsi)*voxelsize
[x, y] = cp.meshgrid(x, x)

ker2=-1j*np.pi*2*distance/wavelength*np.exp(1j*np.pi*(x**2+y**2)/(wavelength*distance))/1024**4#/(2*n)**4/2
# ker2=-1j/(distance*wavelength)*np.exp(1j*np.pi*(x**2+y**2)/(wavelength*distance))/n**3*2#/1024**4#/(2*n)**4/2
# ker2*=16
#*1/(wavelength*np.abs(distance))*
# ker2/=16
# ker2=ker2/(npsi)**4

v = cp.ones(2*npsi)   
vv=cp.linspace(0,1,size_v)
vv = vv**5*(126-420*vv+540*vv**2-315*vv**3+70*vv**4)
v[:npsi-size_v-size_1] = 0
v[npsi-size_v-size_1:npsi-size_1] = vv#cp.sin(cp.linspace(0,1,size_v)*cp.pi/2)        
v[-npsi:] = v[:npsi][::-1]
v = cp.outer(v,v)

ker2 *= v
ker2 = ker2.astype('complex64')
delta = cp.zeros([2*npsi,2*npsi],dtype='complex64')
delta[npsi,npsi]=1

fker = cp.fft.fftshift(cp.fft.fft2(cp.fft.fftshift(ker)))
fker2 = cp.fft.fftshift(cp.fft.fft2(cp.fft.fftshift(ker2)))
fdelta = cp.fft.fftshift(cp.fft.fft2(cp.fft.fftshift(delta)))
fd1 = cp.fft.fftshift(cp.fft.ifft2(cp.fft.fftshift(fdelta*fker)))
fd2 = cp.fft.fftshift(cp.fft.ifft2(cp.fft.fftshift(fdelta*fker2)))

fig, axs = plt.subplots(2, 1, figsize=(15, 5))
axs[0].set_title('abs')
axs[0].plot(np.abs(fd1[npsi,npsi-512:npsi+512].get()),label='freq')
axs[0].plot(np.abs(fd2[npsi,npsi-512:npsi+512].get()),label='space')
    
axs[1].plot(np.real(fd1[npsi,npsi-512:npsi+512].get()),label='freq')
axs[1].plot(np.real(fd2[npsi,npsi-512:npsi+512].get()),label='space')

a = cp.mean(cp.abs(fd1[npsi,npsi-512:npsi+512]))
b = cp.mean(cp.abs(fd2[npsi,npsi-512:npsi+512]))
print(a,b,a/b)

# mshow_complex(fd1,True)
# mshow_complex(fd2,True)

